# Part1
导入必要函数库，初始化GEE

In [ ]:
import ee
import geemap
import pandas as pd

In [ ]:
# geemap.set_proxy(port=7897)
geemap.ee_initialize()

In [ ]:
Map = geemap.Map()
Map

# Part2
定义研究区，筛选Landsat影像并进行预处理

In [ ]:
# 定义研究区（鄂托克旗边界）
shapefile = "./鄂托克旗.json"
featureCol = geemap.geojson_to_ee(shapefile)
Map.addLayer(featureCol,{},'边界')

# 设置地图中心
region = ee.Geometry.Rectangle([106.6833, 38.3, 108.9, 40.1833])
Map = geemap.Map(center=[39.25, 107.8], zoom=8)

In [ ]:
# 导入Landsat 8影像，并筛选无云影像
def applyScaleFactors(image):
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
    return image.addBands(opticalBands, None, True) \
        .addBands(thermalBands, None, True)

# QA波段去云
def CloudMask(image):
    cloudShadowBitMask = (1 << 4)
    cloudsBitMask = (1 << 3)
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
        .And(qa.bitwiseAnd(cloudsBitMask).eq(0))
    return image.updateMask(mask) \
        .copyProperties(image) \
        .copyProperties(image, ["system:time_start"])

In [ ]:
# 导入Landsat数据并裁剪
landsatCol = (ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
           .filterDate('2020-07-01', '2020-08-31')
           .filterBounds(region)
           .filter(ee.Filter.lt('CLOUD_COVER', 30))
           .map(CloudMask)
           .map(applyScaleFactors))
landsat = ee.ImageCollection(landsatCol).median()

# Part3
导入NDVI、高程、坡度、土地利用、土壤基质数据并重分类

In [ ]:
# 计算NDVI并按区间分级
ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
ndvi_classified = ndvi.expression(
    "b('NDVI') < 0 ? 9 : " +       # 第一级 [-1, 0], 赋值为9
    "b('NDVI') < 0.25 ? 7 : " +    # 第二级 [0, 0.25], 赋值为7
    "b('NDVI') < 0.5 ? 5 : " +     # 第三级 [0.25, 0.5], 赋值为5
    "b('NDVI') < 0.75 ? 3 : 1"     # 第四级 [0.5, 0.75], 赋值为3, 第五级 [0.75, 1], 赋值为1
)

In [ ]:
# 导入DEM并分级
dem = ee.Image("USGS/SRTMGL1_003")
elevation_classified = dem.expression(
    "b('elevation') > 2153 ? 9 : " +
    "b('elevation') > 1950 ? 7 : " +
    "b('elevation') > 1700 ? 5 : " +
    "b('elevation') > 1500 ? 3 : 1"
)

In [ ]:
# 计算坡度并分级
slope = ee.Terrain.slope(dem)
slope_classified = slope.expression(
    "b('slope') > 35 ? 9 : " +
    "b('slope') > 25 ? 7 : " +
    "b('slope') > 15 ? 5 : " +
    "b('slope') > 5 ? 3 : 1"
)

In [ ]:
# 导入CLCD土地利用分类数据
imgPathGLB30 = 'users/studyroomGEE/LULC_Dataset/LULC_HuangXin/CLCD_v01_2020'
land_cover = ee.Image(imgPathGLB30)

# 将土地利用数据进行重分类
# 原始类别 [1:耕地, 2:林地, 3:灌丛, 4:草地, 5:水域, 6:冰/雪, 7:荒地，8：不透水面，9：湿地]
# 重分类[1：耕地，2：园地，3：林地，4：草地，5：水域，6：建设用地，7：未利用地]
land_cover_reclassified = land_cover.remap(
    [1, 2, 3, 4, 5, 9, 8, 6, 7],  # 原始类别值
    [1, 3, 2, 4, 5, 5, 6, 7, 7]   # 重分类后的值
)

# 土地利用敏感性分级
land_cover_sensitivity = land_cover_reclassified.remap(
    [7, 5, 3, 4, 1, 6],   # 重分类的值
    [9, 7, 7, 5, 3, 1]     # 敏感性赋值
)

In [ ]:
# 导入土壤基质数据
soil_texture = ee.Image(
    "OpenLandMap/SOL/SOL_CLAY-WFRACTION_USDA-3A1A1A_M/v02").select('b0')

# 将土壤基质数据重分类为：沙质、砾质、壤质、粘质和基岩
# 基于粘土含量分类:
# 0-10: 沙质, 10-20: 砾质, 20-40: 壤质, 40-60: 粘质, 60以上: 基岩
soil_texture_reclassified = soil_texture.expression(
    "b('b0') < 10 ? 9 : " +        # 沙质
    "b('b0') < 20 ? 7 : " +        # 砾质
    "b('b0') < 40 ? 5 : " +        # 壤质
    "b('b0') < 60 ? 3 : 1"         # 粘质和基岩
)

# Option
数据可视化

In [ ]:
# Landsat可视化参数
landsat_vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],  # 使用Landsat的RGB波段
    'min': 0,
    'max': 0.3,
    'gamma': 1.4
}

# 重分类可视化参数
vis_params = {
    'min':1,
    'max':9,
    'palette': ['brown', 'orange', 'yellow', 'lightgreen', 'green']
}

In [ ]:
# 可视化Landsat影像
Map.addLayer(landsat, landsat_vis_params, 'Landsat 8 (2020)')

# 可视化高程
Map.addLayer(dem, {'min': 0, 'max': 3000}, 'DEM')

# 可视化坡度
Map.addLayer(slope, {'min': 0, 'max': 45}, 'Slope')

# 可视化土地利用
Map.addLayer(land_cover_sensitivity, vis_params, 'Reclassified Land Cover')

# 可视化土壤基质
Map.addLayer(soil_texture_reclassified, vis_params, 'Soil Texture')

# 可视化重分类NDVI
Map.addLayer(ndvi_classified, vis_params, 'NDVI Classification')

# 可视化重分类高程
Map.addLayer(elevation_classified, vis_params, 'Elevation Classification')

# 可视化重分类坡度
Map.addLayer(slope_classified, vis_params, 'Slope Classification')

Map.addLayer(featureCol, {}, '边界')
Map

# Part4
敏感性计算

In [ ]:
# 将所有因子合成一个多波段影像
combined_image = ndvi_classified.rename('NDVI')\
    .addBands(elevation_classified.rename('Elevation'))\
    .addBands(slope_classified.rename('Slope'))\
    .addBands(land_cover_sensitivity.rename('LandCover'))\
    .addBands(soil_texture_reclassified.rename('SoilTexture'))

In [ ]:
# 添加合成图像到地图
Map.addLayer(combined_image.clip(featureCol),{},"combine")
Map

In [ ]:
# 使用指定的权重进行加权计算生态敏感性
sensitivity = combined_image.expression(
    "(Elevation * 0.0722) + (Slope * 0.2429) + (NDVI * 0.1256) + (LandCover * 0.4683) + (SoilTexture * 0.0900)",
    {
        'Elevation': combined_image.select('Elevation'),
        'Slope': combined_image.select('Slope'),
        'NDVI': combined_image.select('NDVI'),
        'LandCover': combined_image.select('LandCover'),
        'SoilTexture': combined_image.select('SoilTexture')
    }
).rename('sensitivity')

# Part 5
敏感性分级与制图

In [ ]:
# 计算分位数（五分位数）
quantiles = sensitivity.reduceRegion(
    reducer=ee.Reducer.percentile([20, 40, 60, 80]),
    geometry=region,
    scale=30,
    maxPixels=1e9
).values()

# 获取分位数的实际值
q20, q40, q60, q80 = quantiles.getInfo()

# 使用分位数进行敏感性分级
classified_sensitivity = sensitivity.expression(
    "(sensitivity <= q20) ? 1 : " +   # 非敏感
    "(sensitivity <= q40) ? 2 : " +   # 轻度敏感
    "(sensitivity <= q60) ? 3 : " +   # 中度敏感
    "(sensitivity <= q80) ? 4 : 5",   # 极高度敏感
    {
        'sensitivity': sensitivity,
        'q20': q20,
        'q40': q40,
        'q60': q60,
        'q80': q80
    }
)

In [ ]:
# 设置分级可视化参数
sensitivity_vis = {
    'min': 1,
    'max': 5,
    'palette': ['darkgreen', 'lightgreen', 'yellow', 'orange', 'red'],
    'legend': {
        'Non-sensitive': 'darkgreen',
        'Slightly sensitive': 'lightgreen',
        'Moderately sensitive': 'yellow',
        'Highly sensitive': 'orange',
        'Extremely sensitive': 'red'
    }
}

In [ ]:
# 添加图层到地图
Map.addLayer(featureCol, {}, '边界')
Map.addLayer(classified_sensitivity.clip(featureCol), sensitivity_vis,
             'Classified Sensitivity Map')
# 显示地图
Map

# Option
导出图像到本地

In [ ]:
# 导出生态敏感性图
sensitivity_image = sensitivity.clip(featureCol)
geemap.download_ee_image(
    sensitivity_image,
    filename = 'sensitivity.tif',
    scale = 30,
    crs = "EPSG:4326",
    region = region
)